In [1]:
import pandas as pd
import numpy as np

In [2]:
cd miniproject2

[WinError 2] 지정된 파일을 찾을 수 없습니다: 'miniproject2'
C:\Users\LG\Documents\ais7\miniproject2


In [3]:
visitordf1 = pd.read_csv("data/raw/2018_월별_지역.csv", encoding="cp949")

In [4]:
visitordf2 = pd.read_csv("data/raw/2019_월별_지역.csv", encoding="cp949")

In [5]:
visitordf = pd.concat([visitordf1, visitordf2])

In [6]:
visitordf["기준년월"] = visitordf["기준년월"].astype(str)

In [7]:
visitordf = visitordf.rename(columns={'지자체': '광역지자체', '전년 대비 방문자수 증감률' : '전년대비 방문자증감률'})

In [8]:
visitordf["방문자 구분"] = visitordf["방문자 구분"].str[:3]

In [9]:
visitordf = visitordf.drop(columns=["전년도 방문자수"])

In [10]:
visitordf = visitordf.groupby(by=["기준년월", "광역지자체", "방문자 구분"]).sum()

In [11]:
visitordf = visitordf.unstack(2)

In [12]:
visitordf["외국인 비율", "외국인"] = visitordf["방문자수", "외국인"] / visitordf["방문자수", "외부방"] * 100

In [13]:
visitordf = visitordf.stack(0, dropna=True)["외국인"]

In [14]:
visitordf = visitordf.unstack(2)

In [15]:
visitordf

방문자수     외국인 비율  전년대비 방문자증감률
기준년월   광역지자체                                    
201801 강원도       95574.0   0.926828          0.0
       경기도      374434.0   0.862771          0.0
       경상남도      86978.0   0.774812          0.0
       경상북도      59444.0   0.512048          0.0
       광주광역시     15493.0   0.304022          0.0
...                  ...        ...          ...
201912 전라남도     189828.0   2.076962         35.6
       전라북도      70202.0   0.993265         29.8
       제주특별자치도  554198.0  11.004472         46.1
       충청남도     135321.0   1.184867         26.7
       충청북도      58522.0   0.761488         24.3

[408 rows x 3 columns]

In [31]:
year_list = ["2018", "2019"]
month_list = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]

In [32]:
reservedf = pd.DataFrame()

for year in year_list:
    for month in month_list:
        tempdf = pd.read_csv("data/raw/"+year+month+"_관광소비.csv", encoding="cp949")
        tempdf["기준년월"] = year+month
        reservedf = pd.concat([reservedf, tempdf])

In [33]:
reservedf = reservedf.drop(columns=["전년동기 소비액", "전년대비 소비증감률"])

In [34]:
reservedf

,광역지자체,소비액,기준년월
0,서울특별시,220579456.0,201801
1,부산광역시,20497921.0,201801
2,대구광역시,1986504.0,201801
3,인천광역시,36395462.0,201801
4,광주광역시,916812.0,201801
...,...,...,...
12,전라북도,1207963.0,201912
13,전라남도,512962.0,201912
14,경상북도,1419722.0,201912
15,경상남도,3537608.0,201912


In [63]:
df = pd.merge(visitordf,reservedf, how='inner',on=['기준년월', '광역지자체'])

In [64]:
df['인당 소비액'] = df['소비액'] / df['방문자수']

In [65]:
df['전월대비 방문자증감률'] = 0
df['전년대비 인당소비증감률'] = 0

In [66]:
for i in range(17, 408):
    df['전월대비 방문자증감률'].iloc[i] = ((df['방문자수'].iloc[i]/df['방문자수'].iloc[i-17]) -1) *100

C:\Users\LG\AppData\Local\Temp\ipykernel_12076\2870068271.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['전월대비 방문자증감률'].iloc[i] = ((df['방문자수'].iloc[i]/df['방문자수'].iloc[i-17]) -1) *100


In [67]:
for i in range(0, 204):
    df['전년대비 인당소비증감률'].iloc[i+204] = ((df['인당 소비액'].iloc[i+204]/df['인당 소비액'].iloc[i]) -1) *100

C:\Users\LG\AppData\Local\Temp\ipykernel_12076\211470761.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['전년대비 인당소비증감률'].iloc[i+204] = ((df['인당 소비액'].iloc[i+204]/df['인당 소비액'].iloc[i]) -1) *100


In [68]:
df['방문자수'] = pd.to_numeric(df['방문자수'], downcast="unsigned")
df['소비액'] = pd.to_numeric(df['소비액'], downcast="unsigned")

In [69]:
df['전월대비 방문자증감률']

0       0.000000
1       0.000000
2       0.000000
3       0.000000
4       0.000000
         ...    
403    -3.830020
404   -30.328894
405     5.556296
406    -7.572674
407   -17.290406
Name: 전월대비 방문자증감률, Length: 408, dtype: float64

In [70]:
df['전년대비 인당소비증감률']

0       0.000000
1       0.000000
2       0.000000
3       0.000000
4       0.000000
         ...    
403   -33.601079
404   -16.248179
405   -53.586654
406   -33.890302
407   -31.198008
Name: 전년대비 인당소비증감률, Length: 408, dtype: float64

In [71]:
df = df[['기준년월', '광역지자체', '방문자수', '전월대비 방문자증감률', '전년대비 방문자증감률', '외국인 비율', '소비액', '인당 소비액', '전년대비 인당소비증감률']]

In [72]:
df.to_csv("data/지역별_추이.csv", encoding="cp949", index=False)

In [73]:
pd.read_csv("data/지역별_추이.csv", encoding="cp949")

,기준년월,광역지자체,방문자수,전월대비 방문자증감률,전년대비 방문자증감률,외국인 비율,소비액,인당 소비액,전년대비 인당소비증감률
0,201801,강원도,95574,0.000000,0.0,0.926828,3056657,31.982098,0.000000
1,201801,경기도,374434,0.000000,0.0,0.862771,22074410,58.954075,0.000000
2,201801,경상남도,86978,0.000000,0.0,0.774812,2939692,33.798110,0.000000
3,201801,경상북도,59444,0.000000,0.0,0.512048,1184495,19.926233,0.000000
4,201801,광주광역시,15493,0.000000,0.0,0.304022,916812,59.175886,0.000000
...,...,...,...,...,...,...,...,...,...
403,201912,전라남도,189828,-3.830020,35.6,2.076962,512962,2.702246,-33.601079
404,201912,전라북도,70202,-30.328894,29.8,0.993265,1207963,17.206960,-16.248179
405,201912,제주특별자치도,554198,5.556296,46.1,11.004472,11940389,21.545348,-53.586654
406,201912,충청남도,135321,-7.572674,26.7,1.184867,1506580,11.133379,-33.890302
